In [1]:
import pandas as pd
import numpy as np
import os
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm

In [2]:
ENTREPOT_PATH = '~/Bureau/Datagrosyst/data_entrepot_&_outils/'

# synth = pd.read_csv(ENTREPOT_PATH + 'synthetise.csv')
conx = pd.read_csv(ENTREPOT_PATH + 'connection_synthetise.csv')
noeud = pd.read_csv(ENTREPOT_PATH + 'noeuds_synthetise.csv')

In [3]:
conx = conx[['id', 'frequence_source','culture_absente','source_noeuds_synthetise_id','cible_noeuds_synthetise_id']]\
    .rename(columns={'id' : 'conx_id',
                     'frequence_source' : 'freq',
                     'culture_absente' : 'abs',
                     'source_noeuds_synthetise_id' : 'nd_prec',
                     'cible_noeuds_synthetise_id' : 'nd_suiv'})
noeud = noeud[['id', 'rang', 'fin_cycle','memecampagne_noeudprecedent', 'synthetise_id']]\
    .rename(columns={'id' : 'nd_id',
                     'fin_cycle' : 'end',
                     'rang' : 'rang',
                     'memecampagne_noeudprecedent' : 'sameyear',
                     'synthetise_id' : 'synth_id'})

In [ ]:
check_if_good_schema = [
    # En dessous : ne doit pas passer le filtre

    'fr.inra.agrosyst.api.entities.practiced.PracticedSystem_6d19dd88-dcc4-4a41-8d45-068fbcd4f88c', # end node qui continue
    'fr.inra.agrosyst.api.entities.practiced.PracticedSystem_df56abb7-1a39-4686-bb03-cd85441fc9f6', # commence par 2 noeud
    'fr.inra.agrosyst.api.entities.practiced.PracticedSystem_edf4f144-e853-453b-b4e7-fef8f3ebdb99', # trou dans le chemin
    
    'fr.inra.agrosyst.api.entities.practiced.PracticedSystem_dc9eca9f-8b72-4f1e-b791-790ca4488a60', # full sameyear + qlq abs
    # devrait etre ok..... Oops monsieur a oublié de sommer les sorties du fictif à 100% (99.9%)

    # En dessous : Doit passer le 1° filtre

    'fr.inra.agrosyst.api.entities.practiced.PracticedSystem_592b3792-8ad0-4213-ab98-a948f444ed04', # chemin entierment absent. PASSE CE PREMIER FILTRE ! ne passera pas celui après la fonction principale !!!!!!!!!!!!!!!!!!!!!!
    'fr.inra.agrosyst.api.entities.practiced.PracticedSystem_41ae9d22-5515-44d8-9a7f-8254c42149eb', # ok passe le filtre, classique
    'fr.inra.agrosyst.api.entities.practiced.PracticedSystem_445dd407-58f6-403e-8dd6-8352166a0131', # trou dans le chemin MAIS sameyear dans les trou donc ok !

    # En dessous : ok mais exemple pour la fonction principale
    
    'fr.inra.agrosyst.api.entities.practiced.PracticedSystem_d4a1b64c-afa0-440f-92e1-30a483871ab4', # bcp de sameyear
    'fr.inra.agrosyst.api.entities.practiced.PracticedSystem_61f75804-3823-4fae-9ce1-82bfa3d7e41e', # CBO avec abs dans sameyear
    'fr.inra.agrosyst.api.entities.practiced.PracticedSystem_91611752-8cd2-42f1-b19f-97186597ab64', # CBO avec abs dans sameyear
    'fr.inra.agrosyst.api.entities.practiced.PracticedSystem_2f68d851-cb30-402f-bfc6-b0abf37c49a8' # classique avec des abs et sameyear mais pas intriqués
]


noeud = noeud.loc[noeud['synth_id'].isin(check_if_good_schema)]
conx = conx.loc[(conx['nd_prec'].isin(noeud['nd_id'])) | (conx['nd_suiv'].isin(noeud['nd_id'])),]

In [5]:
df = conx.merge(noeud[['nd_id','rang','end']].add_suffix('_prec'), left_on='nd_prec', right_on='nd_id_prec')\
    .drop('nd_id_prec', axis=1)
df = df.merge(noeud.add_suffix('_suiv'), left_on='nd_suiv', right_on='nd_id_suiv').\
    rename(columns={'synth_id_suiv' : 'synth_id'})\
        .drop('nd_id_suiv', axis=1)

In [6]:
def number_node_rank0(dfgrp):
    return len(dfgrp.loc[dfgrp['rang']==0,'rang'])

def number_node_end(dfgrp):
    return len(dfgrp.loc[dfgrp['end']=='t','end'])

def noeud_end_on_rank0(dfgrp):
    return any(dfgrp.loc[dfgrp['end']=='t','rang']==0)

noeud_test = noeud.groupby('synth_id').apply(
    lambda dfgrp: pd.Series({
        'nb_noeud_de_rang1': number_node_rank0(dfgrp),
        'nb_noeud_finaux' : number_node_end(dfgrp),
        'noeud_finaux_en_rang1' : noeud_end_on_rank0(dfgrp)
        }), include_groups=False).copy()

In [7]:
# noeuds sans connexions avant ou après
print(len(noeud.loc[~((noeud['nd_id'].isin(conx['nd_prec'])) & (noeud['nd_id'].isin(conx['nd_suiv']))),'synth_id']))
# on tague les synthétisé sans aucun noeud de rang 1
print(len(noeud_test.loc[noeud_test['nb_noeud_de_rang1']==0,'nb_noeud_de_rang1']))
# on tague les synthétisé avec plus de 1 noeud de rang 1
print(len(noeud_test.loc[noeud_test['nb_noeud_de_rang1']>1,'nb_noeud_de_rang1']))
# on tague les synthétisé avec aucun noeud terminal
print(len(noeud_test.loc[noeud_test['nb_noeud_finaux']==0,'nb_noeud_finaux']))
# on tague les synthétisé avec au moins 1 noeud terminal en rang 1
print(len(noeud_test.loc[noeud_test['noeud_finaux_en_rang1']>0,'noeud_finaux_en_rang1']))

list_bad_synth = list(noeud_test.loc[(noeud_test['nb_noeud_de_rang1']==0) | \
                                (noeud_test['nb_noeud_de_rang1']>1) | \
                                (noeud_test['nb_noeud_finaux']==0) | \
                                (noeud_test['noeud_finaux_en_rang1']>0)].index)
list_bad_synth = list_bad_synth + list(noeud.loc[~((noeud['nd_id'].isin(conx['nd_prec'])) & (noeud['nd_id'].isin(conx['nd_suiv']))),'synth_id'])

# Supprimer les rang entierement vide
def get_hole_in_rotation(series):
    ranks_theo = pd.Series(range(min(series)+1, max(series)))
    if all(ranks_theo.isin(series)) == False :
        return list(ranks_theo.loc[~(ranks_theo.isin(series))])

test_empty_rank = noeud[['rang','synth_id']].groupby('synth_id').agg(get_hole_in_rotation).reset_index()
test_empty_rank = test_empty_rank.rename(columns={'rang':'empty_ranks'})
test_empty_rank = test_empty_rank.loc[test_empty_rank['empty_ranks'].notna()] # 75 synth avec rang full vide

list_bad_synth = list_bad_synth + list(test_empty_rank['synth_id'])

0
0
1
0
0


In [9]:
# somme de sortie du noeuf doit faire 100%
def concat_unique_sorted_txt(series):
    cleaned = series.dropna().unique().copy()
    return '_&_'.join(sorted(cleaned))

test_100_exit = df[['synth_id','freq','nd_prec']].groupby('nd_prec').agg({
    'synth_id' : concat_unique_sorted_txt,
    'freq' : 'sum'})
test_100_exit = test_100_exit.loc[test_100_exit['freq'] != 100,]

list_bad_synth = list_bad_synth + list(test_100_exit['synth_id'])

In [10]:
list_bad_synth

['fr.inra.agrosyst.api.entities.practiced.PracticedSystem_df56abb7-1a39-4686-bb03-cd85441fc9f6',
 'fr.inra.agrosyst.api.entities.practiced.PracticedSystem_dc9eca9f-8b72-4f1e-b791-790ca4488a60']

In [9]:
# pas de connexion qui sont à 0% (apparament mais je sias pas pk)
# on fait attention aux connexions inférieurs à 0.5%

test_cnx_0 = df.loc[df['freq'] < 0.5,].copy()

list_bad_synth = list_bad_synth + list(test_cnx_0['synth_id'])

In [10]:
# noeud suivant est forcement sur le rang suivant (pas de 'trou') 
# attention si le rang qui a un trou dans le chemin n'est fait que de culture dérobée et que le noeud suivant n'est pas une dérobée

# OU au moins sur un rang supérieur 
# test = df.loc[(df['rang_prec'] > df['rang_suiv'] ) & (df['end_prec'] == 'f'),] # apparement impossible vu que 0 fois

test_hole_in_path = df.loc[(df['rang_prec'] != (df['rang_suiv']-1) ) & (df['end_prec'] == 'f'),].copy()
test_hole_in_path['empty_rank_are_catch_crop'] = ''

for idx, row in test_hole_in_path.iterrows() :
    empty_rank_list = list(range(row['rang_prec']+1, row['rang_suiv']))
    if list(noeud.loc[(noeud['synth_id']==row['synth_id']) & (noeud['rang'].isin(empty_rank_list)), 'sameyear']=='t') == []:
        test_hole_in_path.at[idx,'empty_rank_are_catch_crop'] = 'empty_rank'
    elif (row['sameyear_suiv'] == 'f') & (all(list(noeud.loc[(noeud['synth_id']==row['synth_id']) & (noeud['rang'].isin(empty_rank_list)), 'sameyear'] == 't'))) & (all(list(noeud.loc[(noeud['synth_id']==row['synth_id']) & (noeud['rang']==row['rang_suiv']), 'sameyear'] == 'f'))) \
    | \
    (row['sameyear_suiv'] == 'f') & (all(list(noeud.loc[(noeud['synth_id']==row['synth_id']) & (noeud['rang'].isin(empty_rank_list)), 'sameyear'] == 'f'))) & (all(list(noeud.loc[((noeud['synth_id']==row['synth_id']) & (noeud['rang']==row['rang_suiv'])) & (noeud['nd_id']!=row['nd_suiv']), 'sameyear'] == 't'))) :
        test_hole_in_path.at[idx,'empty_rank_are_catch_crop'] = 'ok'
    else :
        test_hole_in_path.at[idx,'empty_rank_are_catch_crop'] = 'hole_in_path'


test_hole_in_path = test_hole_in_path.groupby('synth_id').agg({'empty_rank_are_catch_crop': concat_unique_sorted_txt}).reset_index()

list_bad_synth = list_bad_synth + list(test_hole_in_path.loc[test_hole_in_path['empty_rank_are_catch_crop'] != 'ok', 'synth_id'])

In [11]:
# Noeud terminaux qui ne boucle pas entierement sur un noeud de premier rang
end_node_continue = list(df.loc[(df['end_prec'] == 't') & df['rang_suiv'] != 0, 'synth_id'])

list_bad_synth = list_bad_synth + end_node_continue

In [12]:
list_bad_synth = list(set(list_bad_synth))

cx = df.loc[~(df['synth_id'].isin(list_bad_synth))].set_index('conx_id').copy()
nd = noeud.loc[~(noeud['synth_id'].isin(list_bad_synth))].set_index('nd_id').copy()

list_good_synth = tuple(set(nd['synth_id']))


In [13]:
list_bad_synth

['fr.inra.agrosyst.api.entities.practiced.PracticedSystem_df56abb7-1a39-4686-bb03-cd85441fc9f6',
 'fr.inra.agrosyst.api.entities.practiced.PracticedSystem_6d19dd88-dcc4-4a41-8d45-068fbcd4f88c',
 'fr.inra.agrosyst.api.entities.practiced.PracticedSystem_9791de7e-ccc4-4ecb-9cbb-a93273022ce8',
 'fr.inra.agrosyst.api.entities.practiced.PracticedSystem_edf4f144-e853-453b-b4e7-fef8f3ebdb99',
 'fr.inra.agrosyst.api.entities.practiced.PracticedSystem_dc9eca9f-8b72-4f1e-b791-790ca4488a60']

In [14]:
list_good_synth

('fr.inra.agrosyst.api.entities.practiced.PracticedSystem_91611752-8cd2-42f1-b19f-97186597ab64',
 'fr.inra.agrosyst.api.entities.practiced.PracticedSystem_445dd407-58f6-403e-8dd6-8352166a0131',
 'fr.inra.agrosyst.api.entities.practiced.PracticedSystem_41ae9d22-5515-44d8-9a7f-8254c42149eb',
 'fr.inra.agrosyst.api.entities.practiced.PracticedSystem_61f75804-3823-4fae-9ce1-82bfa3d7e41e',
 'fr.inra.agrosyst.api.entities.practiced.PracticedSystem_d4a1b64c-afa0-440f-92e1-30a483871ab4',
 'fr.inra.agrosyst.api.entities.practiced.PracticedSystem_2f68d851-cb30-402f-bfc6-b0abf37c49a8',
 'fr.inra.agrosyst.api.entities.practiced.PracticedSystem_592b3792-8ad0-4213-ab98-a948f444ed04')

In [ ]:
# Fonction pour trouver tous les chemins possibles avec DFS (itératif pour éviter la récursion trop profonde)
def trouver_chemins(graphe, debut, fins):
    stack = [(debut, [])]  # (node actuel, chemin parcouru)
    chemins = []

    while stack:
        node, chemin = stack.pop()
        chemin.append(node)

        if node in fins:
            chemins.append(list(chemin))
            continue

        for voisin in graphe.get(node, []):
            if voisin not in chemin:  # Éviter les cycles
                stack.append((voisin, chemin[:]))  # Copie du chemin actuel

    return chemins


final_data = pd.DataFrame()


def process_sy(sy):

    # Construire le graphe des connexions sous forme de dictionnaire
    graphe = {}
    connexions = {}

    for idx, row in cx.iterrows():
        nd_prec, nd_suiv, conx_id, abs_value, freq = row['nd_prec'], row['nd_suiv'], idx, row['abs'], row['freq']
        
        if nd_prec not in graphe:
            graphe[nd_prec] = []
        graphe[nd_prec].append(nd_suiv)

        # Stocker les connexions avec leurs IDs et attributs pour un accès rapide
        connexions[(nd_prec, nd_suiv)] = {
            'conx_id': conx_id,
            'abs': abs_value,
            'freq': freq
        }

    # Trouver le premier nœud et les nœuds finaux
    first_node = nd.loc[(nd['rang'] == 0) & (nd['synth_id'] == sy)].index.item()
    end_nodes = set(nd.loc[(nd['end'] == 't') & (nd['synth_id'] == sy)].index)

    # Obtenir tous les chemins
    chemins_possibles = trouver_chemins(graphe, first_node, end_nodes)

    #  Associer chaque connexion aux chemins où elle apparaît, en comptant abs='t', sameyear='t', et calculant le poids du chemin
    df_couples_connexion_chemins = []
    df_chemins = []

    for chemin in chemins_possibles:

        poid_chemin = 1
        groupe_id = 0
        groupes_sameyear = {}

        # Déterminer les groupes sameyear chemin par chemin
        for i, node3 in enumerate(chemin):
            if i == 0 or nd.loc[node3, 'sameyear'] == 'f':
                groupe_id += 1  # Nouveau groupe
            groupes_sameyear[node3] = groupe_id  # Associer le nœud à son groupe
        # Si le premier noeud est en sameyear on associe son groupe à toutes les connexions qui ont le meme groupe que le dernier neoud
        if nd.loc[chemin[0], 'sameyear'] == 't':
            list_node_same_grp_as_last = [key for key, value in groupes_sameyear.items() if value == groupes_sameyear[chemin[-1]]]
            for key in list_node_same_grp_as_last: groupes_sameyear[key] = groupes_sameyear[chemin[0]]

        # Calculer le poids du chemin (produit) et Ajouter les infos de couples connexions/chemins
        for i in range(len(chemin) - 1):
            nd_prec, nd_suiv = chemin[i], chemin[i + 1]
            if (nd_prec, nd_suiv) in connexions:
                poid_chemin *= connexions[(nd_prec, nd_suiv)]['freq'] / 100  # Convertir en probabilité
                df_couples_connexion_chemins.append({
                    'connexion_id': connexions[(nd_prec, nd_suiv)]['conx_id'],
                    'chemin_id': chemin,
                    'groupe_sameyear': groupes_sameyear[nd_suiv],  # La connexion prend le groupe du noeud suivant
                    'abs': connexions[(nd_prec, nd_suiv)]['abs']
                })
        # Ajout des connexions des nœuds terminaux vers le premier nœud
        if chemin[-1] in end_nodes and (chemin[-1], chemin[0]) in connexions:
            poid_chemin *= connexions[(chemin[-1], chemin[0])]['freq'] / 100
            df_couples_connexion_chemins.append({
                'connexion_id': connexions[(chemin[-1], chemin[0])]['conx_id'],
                'chemin_id': chemin,
                'groupe_sameyear': groupes_sameyear[chemin[0]],  # La connexion prend le groupe du premier noeud
                'abs': connexions[(chemin[-1], chemin[0])]['abs']
            })
        
        # Nombre d'année (soit le nombre de groupe sameyear après avoir enlever les connexions absentes)
        nb_annee = len(set([entry['groupe_sameyear'] for entry in df_couples_connexion_chemins if \
                            (entry['abs'] == 'f') & (entry['chemin_id'] == chemin)]))
        if nb_annee == 0 : nb_annee = 1
        # Ajouter les infos des chemins, dont le poids des chemins après recalcul sans conx absente
        df_chemins.append({
            'chemin_id': chemin,
            'pd_chem' : poid_chemin,
            'poid_standard_cnx_wo_abs': poid_chemin / nb_annee,
            'synth_id': sy
        })

    # Convertir en DataFrame
    df_chemins = pd.DataFrame(df_chemins)
    df_chemins['chemin_id'] = df_chemins['chemin_id'].astype(str)
    df_couples_connexion_chemins = pd.DataFrame(df_couples_connexion_chemins)
    df_couples_connexion_chemins['chemin_id'] = df_couples_connexion_chemins['chemin_id'].astype(str)
    # Merge chemin sur les couples cx_ch
    df_couples_connexion_chemins = df_couples_connexion_chemins.merge(df_chemins, on = 'chemin_id', how = 'left')

    # Avoir le compte du nombre de connexions active (abs == 'f') ayant le meme chemin ET le meme groupe_sameyear
    nb_grp_sameyear_overall = df_couples_connexion_chemins[df_couples_connexion_chemins['abs'] == 'f'].\
        groupby(['chemin_id', 'groupe_sameyear']).size().reset_index(name='count_grp_sameyear_overall')
    all_df = df_couples_connexion_chemins.merge(nb_grp_sameyear_overall, on=['chemin_id', 'groupe_sameyear'], how='left')

    # Calculer le vrai poids de connexion final (poid_standard_cnx_wo_abs / count_grp_sameyear_overall)
    all_df['connexion_freq'] = all_df['poid_standard_cnx_wo_abs'] / all_df['count_grp_sameyear_overall']

    # Supprimer le poids de connexion des connexions absentes
    all_df.loc[all_df['abs'] == 't','connexion_freq'] = np.nan

    return all_df



# Utilisation de ProcessPoolExecutor avec 80% des cœurs
with ProcessPoolExecutor(max_workers= max(1, int(os.cpu_count() * 0.7)) ) as executor:
    results = list(tqdm(executor.map(process_sy, list_good_synth), total=len(list_good_synth)))

# Concaténation des résultats
final_data = pd.concat(results)

test = final_data.copy()
test = test[['connexion_freq','synth_id']].groupby('synth_id').sum('connexion_freq')
test = test.loc[round(test['connexion_freq'],2) != 1]

final_data = final_data.loc[~(final_data['synth_id'].isin(test.index))]

final_data_conx_level = final_data[['connexion_id','connexion_freq','abs','synth_id']].groupby('connexion_id').agg({
    'connexion_freq' : lambda x : np.nan if all(x.isna()) else sum(x.dropna()),
    'abs' : lambda x : x.unique().item(), # Pas exportée
    'synth_id' : lambda x : x.unique().item() # Pas exportée
})

In [ ]:
final_data.to_csv('/home/administrateur/Bureau/test_conx_chem.csv', index = False)
final_data_conx_level.to_csv('/home/administrateur/Bureau/test_weigth_conx.csv', index = False)
